In [158]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [159]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

In [160]:
data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush.csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    # add queried data (without cost) to initial data
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)

    # data = data.append(queried_data, ignore_index=True)
    return data

data = add_data(data, df_queried)
data = data.reset_index().astype('Float32')

print(data)

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

     index  Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
0      0.0         700.0         7.33    500.899994   78.375298         1.91   
1      1.0         700.0        25.85    577.599976  118.067299         0.94   
2      2.0         700.0    46.669998    615.799988  144.355698         2.86   
3      3.0         700.0    69.550003    778.299988  174.177902         2.22   
4      4.0         700.0    94.330002    900.099976  213.314804        -1.72   
..     ...           ...          ...           ...         ...          ...   
108    8.0   2183.257812    47.490673   1444.360107  650.760376     4.574534   
109    9.0   1896.591187    97.303398   1351.508545  827.040039    -3.222039   
110   10.0   1199.474487   101.517929   1868.645874  393.788116    -0.035194   
111   11.0     903.48175    10.912282    960.143188  602.400879     4.090658   
112   12.0    989.338928    81.168571    798.997742  610.851624     4.238798   

     Intake pressure  Back pressure  In

In [161]:
# Datenaufteilung in Training und Test
from numpy import double


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# X_train = scaler_X.fit_transform(X_train)
# X_test = scaler_X.transform(X_test)

# y_train = scaler_y.fit_transform(y_train)
# y_test = scaler_y.transform(y_test)

x_train = np.array(X_train.values, dtype=float)
x_test = np.array(X_train.values, dtype=float)
y_test = np.array(y_test.values, dtype=float)
y_train = np.array(y_train.values, dtype=float)

# Umwandlung in PyTorch Tensors
X_train = torch.from_numpy(x_train).to(torch.float32)
X_test = torch.from_numpy(x_test).to(torch.float32)
y_train = torch.from_numpy(y_train).to(torch.float32)
y_test = torch.from_numpy(y_test).to(torch.float32)


In [162]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 8)
        self.layer2 = nn.Linear(8, 12)
        self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        #print(x[0])
        #mat = torch.diag(torch.tensor([2, 2, 1, 1, 1, 1, 1, 1])).to(torch.float32)
        #print(mat[0])
        #x = torch.matmul(x, mat)
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [163]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 12

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [164]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/300], Loss: 2108.7363
Epoch [20/300], Loss: 2839.7180
Epoch [30/300], Loss: 2331.1504
Epoch [40/300], Loss: 2025.2147
Epoch [50/300], Loss: 2182.6765
Epoch [60/300], Loss: 1853.4552
Epoch [70/300], Loss: 1149284.0000
Epoch [80/300], Loss: 3080.1206
Epoch [90/300], Loss: 4705.1797
Epoch [100/300], Loss: 1114788.5000
Epoch [110/300], Loss: 4119.9058
Epoch [120/300], Loss: 4079.7297
Epoch [130/300], Loss: 1977.7697
Epoch [140/300], Loss: 436.6842
Epoch [150/300], Loss: 2620.4675
Epoch [160/300], Loss: 10035.5928
Epoch [170/300], Loss: 679.9958
Epoch [180/300], Loss: 888.0767
Epoch [190/300], Loss: 372.7469
Epoch [200/300], Loss: 646.4191
Epoch [210/300], Loss: 602850.7500
Epoch [220/300], Loss: 1277.4412
Epoch [230/300], Loss: 2208.1414
Epoch [240/300], Loss: 1119.9995
Epoch [250/300], Loss: 2904.8701
Epoch [260/300], Loss: 3343.4062
Epoch [270/300], Loss: 39091.6719
Epoch [280/300], Loss: 1542.8361
Epoch [290/300], Loss: 10724.7471
Epoch [300/300], Loss: 1558.4537


In [165]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    #test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    #print(f'Test Loss: {test_loss.item():.4f}')
    # y_pred = scaler_y.inverse_transform(y_pred)
    # y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

tensor([162.6282,   2.7169,  52.8198,   3.4087,  70.2730]):tensor([165.1238,   3.6310, 104.6658,   7.3947,  99.4201])
tensor([ 229.7275,    5.0225,   68.9522, 1581.6304,   95.5289]):tensor([315.4302,   0.7733,  39.2699,   4.2984,  93.8421])
tensor([78.3442,  1.9820, 24.9224, -9.7573, 36.7006]):tensor([134.1714,   3.2451,  65.6674,  12.9136,  74.5443])
tensor([115.6264,   1.6783,  37.3084,   2.9377,  49.9820]):tensor([50.6014,  0.5372, 16.3025,  2.3297, 53.8660])
tensor([179.5125,   3.6901,  58.3207,  -3.6790,  79.1634]):tensor([22.0959,  0.1985,  3.6819,  1.9589, 45.7711])
tensor([ 89.5824,   2.3659,  28.6052, -11.2900,  41.9356]):tensor([94.2330,  2.8329, 38.5090, 10.5363, 59.9567])
tensor([308.4649,   5.5858, 100.9872,   5.9977, 132.5008]):tensor([66.8694,  4.0959, 36.2915, 16.2833, 60.4979])
tensor([150.1138,   3.0975,  48.6105,  -4.2009,  66.5967]):tensor([187.1987,   2.8930,  92.9965,   4.8530, 101.0711])
tensor([124.1373,   4.5674,  39.9935, -27.3403,  60.2959]):tensor([125.6448,

In [166]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_max
    matrix.append(percentage_to_range)
#print(matrix)


In [167]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



4.361164592031426
93.59240848953847
4.6366959124925575
1.6480866286250229
11.485070306361125
0.5477522285149352
26.341257394364117
4.236924677175259
2.6175278430917213
6.822538284905559
3.4420077437287633
32.764727527411694
Mean:
16.041346802353388


In [168]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

# y_pred_t = np.transpose(y_pred)
# y_test_t = np.transpose(y_test)

# # print(y_pred_t.shape)
# # print(y_test_t.shape)
# featurewise_mean = []
# for feature_t, feature_p in zip(y_test_t, y_pred_t):
#     # print(feature_t)
#     # print(feature_p)
#     # print(mean_squared_error(feature_t, feature_p))
#     featurewise_mean.append(mean_squared_error(feature_t, feature_p))

# print(featurewise_mean)

# #print(y_range)
# featurewise_mean_percentage = featurewise_mean / y_max**2
# print("mse per feature in percentage to value range")
# print(featurewise_mean_percentage)

In [169]:
model.eval()
submission_data = pd.read_csv("submission.csv")


#print(submission_data)
#print(submission_data.keys())

submission_keys = ['Speed', 'Load', 'Railpressure', 'Air', 'Angle', 'Pressure1','Pressure2', 'Temperature']
model_keys = ['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']

for sub_key, mod_key in zip(submission_keys, model_keys):
    submission_data = submission_data.rename(columns={sub_key:mod_key})



# input = input.reshape((1,8))
submission_result = []
for submission_datapoint in submission_data.values:
    input_tensor = torch.from_numpy(submission_datapoint).to(torch.float32)
    #print(type(input_tensor))
    submission_result.append(model(input_tensor))


In [170]:

#print(submission_result)
submission_result = torch.from_numpy(np.array([result.detach().numpy() for result in submission_result]))
submission_result_t = torch.transpose(submission_result, 0, 1)
submission_std = []
submission_mean = []

print(submission_result_t.shape)
#print(submission_result)
for feature in submission_result_t:
    submission_std.append(feature.std())
    submission_mean.append(feature.mean())

print(submission_mean)
print(submission_std)

y_std = []
y_mean = []
#print(y)
y_t = np.transpose(np.array(y))

for feature in y_t:
    #print(feature)
    y_std.append(feature.std())
    y_mean.append(feature.mean())


print(y_mean)
print(y_std)

q_mean = np.divide(y_mean, submission_mean)
q_std = np.divide(y_std, submission_std)

print("\nQuotienten von mean und std:\n")
print(q_mean)
print(q_std)


torch.Size([5, 57892])
[tensor(245.4014), tensor(5.4833), tensor(78.2920), tensor(487.2426), tensor(106.2324)]
[tensor(87.5890), tensor(1.7056), tensor(27.7163), tensor(1000.3574), tensor(33.4040)]
[185.93122700885334, 2.0784391177966532, 59.99559786467426, 54.2099076163238, 85.385158066201]
[154.61851890040552, 1.8747206523587878, 40.30833065045906, 515.0800001367423, 29.568229488804505]

Quotienten von mean und std:

[0.75766165 0.37905059 0.7663059  0.11125855 0.80375845]
[1.76527239 1.0991418  1.45431615 0.51489597 0.88517044]
